# Sækjum gildin fyrir núverandi control methods

In [25]:
import sys
import os

os.chdir("/Users/frodibrooks/Desktop/DTU/Thesis/OptimisedHeating/AgentV2")

In [28]:
from pump_env_demands import WdsWithDemand
import pandas as pd
import numpy as np

def get_state_from_input_named(demand: float, pumpgroup_speeds: list[float]) -> pd.DataFrame:
    assert len(pumpgroup_speeds) == 3, "Expected 3 pump group speeds"

    env = WdsWithDemand(demand_pattern=np.array([demand]), use_constant_demand=True)
    env.reset()

    # Set pump group speeds
    for group_idx, speed in enumerate(pumpgroup_speeds):
        for pump_id in env.pumpGroups[group_idx]:
            env.pump_speeds[pump_id] = speed
            env.wds.pumps[pump_id].speed = speed

    # Apply scaled demand
    for junction in env.wds.junctions:
        junction.basedemand = env.demandDict[junction.uid] * demand

    # Solve and compute state
    env.wds.solve()
    env.pump_power()
    env.calculate_pump_efficiencies()

    # Use get_state_named instead of get_state
    named_state = env.get_state_named()

    # Add demand and pump group speeds to state
    named_state['input_demand'] = demand
    for i, speed in enumerate(pumpgroup_speeds):
        named_state[f'pumpgroup_{i+1}_speed'] = speed

    # Return as DataFrame
    return pd.DataFrame([named_state])


In [29]:
df = get_state_from_input_named(1.0, [1.0, 1.1, 1.2])
print(df.T)  # Transpose for better readability


AttributeError: 'WdsWithDemand' object has no attribute 'get_state_named'